In [1]:
import pandas as pd

In [ ]:
test = pd.read_csv('/Users/t2023-m0149/Documents/spartacodingclub/Projects/4. insurance/test.csv')

In [ ]:
sample = pd.read_csv('/Users/t2023-m0149/Documents/spartacodingclub/Projects/4. insurance/sample.csv')

In [ ]:
train = pd.read_csv('/Users/t2023-m0149/Documents/spartacodingclub/Projects/4. insurance/train.csv')

In [5]:
print("전체 행 개수 :", len(train))
print(train.isna().sum())     

전체 행 개수 : 1200000
id                           0
Age                      18705
Gender                       0
Annual Income            44949
Marital Status           18529
Number of Dependents    109672
Education Level              0
Occupation              358075
Health Score             74076
Location                     0
Policy Type                  0
Previous Claims         364029
Vehicle Age                  6
Credit Score            137882
Insurance Duration           1
Policy Start Date            0
Customer Feedback        77824
Smoking Status               0
Exercise Frequency           0
Property Type                0
Premium Amount               0
dtype: int64


## 데이터 전처리_재시도

In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

# 데이터 불러오기
df = pd.read_csv("train.csv")  # 파일 경로에 맞게 수정

# 1. 로그 변환
df['Annual Income'] = np.log1p(df['Annual Income'])

# 2. 결측치 처리
mean_fill = ['Health Score', 'Credit Score', 'Age', 'Vehicle Age']
median_fill = ['Previous Claims']

for col in mean_fill:
    df[col] = df[col].fillna(df[col].mean())

for col in median_fill:
    df[col] = df[col].fillna(df[col].median())

# 3. Label Encoding (범주형)
label_cols = [
    'Marital Status', 'Education Level', 'Policy Type', 'Gender',
    'Smoking Status', 'Number of Dependents', 'Property Type',
    'Exercise Frequency', 'Location'
]

for col in label_cols:
    df[col] = df[col].astype(str)
    df[col] = LabelEncoder().fit_transform(df[col])

# 4. 고유값 제거
df.drop(columns=['id'], inplace=True)

# 5. 날짜형 처리
df['Policy Start Date'] = pd.to_datetime(df['Policy Start Date'], errors='coerce')
df['Policy_Year'] = df['Policy Start Date'].dt.year
df['Policy_Month'] = df['Policy Start Date'].dt.month
df.drop(columns=['Policy Start Date'], inplace=True)

# 6. 다중 클래스 범주형 -> 빈도 인코딩
for col in ['Occupation', 'Customer Feedback']:
    freq_map = df[col].value_counts().to_dict()
    df[col] = df[col].map(freq_map)

# 전처리 완료
print(df.info())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200000 entries, 0 to 1199999
Data columns (total 21 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   Age                   1200000 non-null  float64
 1   Gender                1200000 non-null  int64  
 2   Annual Income         1155051 non-null  float64
 3   Marital Status        1200000 non-null  int64  
 4   Number of Dependents  1200000 non-null  int64  
 5   Education Level       1200000 non-null  int64  
 6   Occupation            841925 non-null   float64
 7   Health Score          1200000 non-null  float64
 8   Location              1200000 non-null  int64  
 9   Policy Type           1200000 non-null  int64  
 10  Previous Claims       1200000 non-null  float64
 11  Vehicle Age           1200000 non-null  float64
 12  Credit Score          1200000 non-null  float64
 13  Insurance Duration    1199999 non-null  float64
 14  Customer Feedback     1122176 non-

In [7]:
import pandas as pd


# 1. Annual Income 결측치 제거
df = df.dropna(subset=['Annual Income'])


# 2. Occupation 결측치를 'Unknown'으로 대체
df['Occupation'] = df['Occupation'].fillna('Unknown')


# 3. Customer Feedback 결측치를 'Unknown'으로 대체
df['Customer Feedback'] = df['Customer Feedback'].fillna('Unknown')


#4.
df.drop('Insurance Duration', axis=1, inplace=True)  # 보험 청구 여부 제거
# 결과 확인
print(df.isnull().sum())


# 저장
df.to_csv("cleaned_train.csv", index=False)




Age                     0
Gender                  0
Annual Income           0
Marital Status          0
Number of Dependents    0
Education Level         0
Occupation              0
Health Score            0
Location                0
Policy Type             0
Previous Claims         0
Vehicle Age             0
Credit Score            0
Customer Feedback       0
Smoking Status          0
Exercise Frequency      0
Property Type           0
Premium Amount          0
Policy_Year             0
Policy_Month            0
dtype: int64


In [9]:
pip install lightgbm

     |████████████████████████████████| 1.6 MB 3.1 MB/s eta 0:00:01
You should consider upgrading via the '/Users/t2023-m0149/Documents/GitHub/4gun4_new_project/alex/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import lightgbm as lgb
# 데이터 불러오기
df = pd.read_csv("train.csv")
# 로그 변환
df['Annual Income'] = np.log1p(df['Annual Income'])
# 결측치 처리
mean_fill = ['Health Score', 'Credit Score', 'Age', 'Vehicle Age']
median_fill = ['Previous Claims']
for col in mean_fill:
    df[col] = df[col].fillna(df[col].mean())
for col in median_fill:
    df[col] = df[col].fillna(df[col].median())
# Label Encoding
label_cols = [
    'Marital Status', 'Education Level', 'Policy Type', 'Gender',
    'Smoking Status', 'Number of Dependents', 'Property Type',
    'Exercise Frequency', 'Location'
]
for col in label_cols:
    df[col] = df[col].astype(str)
    df[col] = LabelEncoder().fit_transform(df[col])
# ID 제거
df.drop(columns=['id'], inplace=True)
# 날짜 처리
df['Policy Start Date'] = pd.to_datetime(df['Policy Start Date'], errors='coerce')
df['Policy_Year'] = df['Policy Start Date'].dt.year
df['Policy_Month'] = df['Policy Start Date'].dt.month
df.drop(columns=['Policy Start Date'], inplace=True)
# Occupation, Customer Feedback → 결측치 처리
df['Occupation'] = df['Occupation'].fillna('Unknown')
df['Customer Feedback'] = df['Customer Feedback'].fillna('Unknown')
# frequency encoding
for col in ['Occupation', 'Customer Feedback']:
    freq_map = df[col].value_counts().to_dict()
    df[col] = df[col].map(freq_map)
# Insurance Duration 결측치 제거
df = df.dropna(subset=['Insurance Duration'])
# 범주형 컬럼 frequency encoding
categorical_cols = [
    'Gender', 'Marital Status', 'Education Level', 'Location',
    'Policy Type', 'Smoking Status', 'Exercise Frequency', 'Property Type'
]
for col in categorical_cols:
    freq_map = df[col].value_counts().to_dict()
    df[col] = df[col].map(freq_map)
# 상관관계 높은 피처 제거
corr_matrix = df.corr(numeric_only=True)
high_corr_features = corr_matrix["Premium Amount"][corr_matrix["Premium Amount"].abs() > 0.6].index.tolist()
high_corr_features = [f for f in high_corr_features if f != "Premium Amount"]
# 로그 타겟 생성
df["Premium Amount"]
# X, y 분리
X = df.drop(columns=high_corr_features + ["Premium Amount"])
y = df["Premium Amount"]
# 데이터 분리
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
# LightGBM Dataset 생성
train_data = lgb.Dataset(X_train, label=y_train)
valid_data = lgb.Dataset(X_valid, label=y_valid)
# LightGBM 파라미터
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'verbosity': -1,
    'boosting_type': 'gbdt',
    'max_depth': 2,
    'num_leaves': 5,
    'min_data_in_leaf': 100,
    'learning_rate': 0.01,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.5,
    'bagging_freq': 5,
    'seed': 42
}
# 학습
model = lgb.train(
    params,
    train_data,
    valid_sets=[train_data, valid_data],
    num_boost_round=3000,
    callbacks=[lgb.early_stopping(stopping_rounds=100), lgb.log_evaluation(100)]
)
# 1. 예측
y_pred_log = model.predict(X_valid)
# 2. clip으로 값 폭주 방지
y_pred_log = np.clip(y_pred_log, a_min=None, a_max=20)
y_valid_log = np.clip(y_valid, a_min=None, a_max=20)
# 3. expm1으로 원상복구
y_pred = np.expm1(y_pred_log)
y_valid_orig = np.expm1(y_valid_log)
# 4. 평가
print(":흰색_확인_표시: 최종 성능:")
print(":압정: MAE:", mean_absolute_error(y_valid_orig, y_pred))
print(":압정: MSE:", mean_squared_error(y_valid_orig, y_pred))
print(":압정: RMSE:", np.sqrt(mean_squared_error(y_valid_orig, y_pred)))
print(":압정: R2:", r2_score(y_valid_orig, y_pred))







ModuleNotFoundError: No module named 'lightgbm'